# **Canais Quânticos - Exemplo 3 - Protocolos Quânticos**

Neste notebook praticaremos como trabalhar com protocolos quânticos que podemos anexar aos nós e que podemos utilizar para executar programas nos nós. Isso nos ajudará a simular mais facilmente cenários mais complexos.

In [1]:
import netsquid as ns
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.components.qprogram import QuantumProgram
from netsquid.nodes import Node
from netsquid.nodes.connections import Connection
from netsquid.components.cchannel import ClassicalChannel
from netsquid.components.qchannel import QuantumChannel
import netsquid.components.instructions as instr
from netsquid.protocols.nodeprotocols import NodeProtocol
from netsquid.protocols.protocol import Signals
import pydynaa

Para praticar, nós iremos reescrever o exemplo anterior usando protocolos para cada nó.

Na sequência, iremos:
- Definir as instruções físicas que queremos utilizar nas memórias quânticas;
- Inicializar os processadores;
- Inicializar os nós;
- Inicializar o canal clássico;
- Conectar os dois nós a esse canal.

In [2]:
phys_instructions3 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3),
    PhysicalInstruction(instr.INSTR_H, duration=1),
    PhysicalInstruction(instr.INSTR_X, duration=1),
    PhysicalInstruction(instr.INSTR_MEASURE, duration=7)]

qproc3a = QuantumProcessor(name="ExampleQProc3a" , num_positions =1, phys_instructions = phys_instructions3)
qproc3b = QuantumProcessor(name="ExampleQProc3b" , num_positions =1, phys_instructions = phys_instructions3)

node3a = Node(name="ExampleNode3a", qmemory=qproc3a, port_names=['cin_3a','cout_3a'])
node3b = Node(name="ExampleNode3b", qmemory=qproc3b, port_names=['cin_3b','cout_3b'])

cchannel3 = ClassicalChannel(name="CChannel_3a23b")

node3a.ports['cout_3a'].connect(cchannel3.ports["send"])
node3b.ports['cin_3b'].connect(cchannel3.ports["recv"])

Em seguida, definimos um programa quântico para o node3a (o mesmo de antes) onde:
- Inicializamos um qubit no slot 0 no node3a;
- Mudamos o estado do qubit para  $|+\rangle = \frac{1}{\sqrt 2}(|0\rangle + |1\rangle)$;
- Medimos o qubit.

In [3]:
class node3aProgram(QuantumProgram):
    default_num_qubits = 1 # Número de qubits
    
    def program(self):
        q0, = self.get_qubit_indices(1) #obtendo os índices para o qubit que estaremos trabalhando
        self.apply(instr.INSTR_INIT, [q0]) # Inicializando o qubit
        
        # conjunto do estado do qubit para h0
        self.apply(instr.INSTR_H, [q0]) # aplicando a porta H no q0
        
        # Agora medindo qubit
        self.apply(instr.INSTR_MEASURE, q0, output_key="m1") # Medindo q0 e armazenando o resultado
        
        yield self.run() # Rodando as instruções agendadas acima

No exemplo 2, executamos o programa diretamente no NodeA. Agora, vamos incorporá-lo dentro de um protocolo. O protocolo primeiro executará o programa e assim que o qubit for medido, ele enviará o resultado da medição para a porta de saída do node3a.

In [4]:
class node3aProtocol(NodeProtocol):
    def run(self):
        node3aprog = node3aProgram()
        
        yield self.node.qmemory.execute_program(node3aprog)
        
        m1, = node3aprog.output["m1"]
        print("measurement outcome", m1)
        self.node.ports["cout_3a"].tx_output(m1)

<br><br>No node3b, queremos inicializar um qubit. Para realizar esta etapa, escrevemos um programa que será executado em node3b

In [5]:
class node3bProgram(QuantumProgram):
    default_num_qubits = 1 # Número de qubits
    
    def program(self):
        q0, = self.get_qubit_indices(1) #obtendo os índices para o qubit que estaremos trabalhando
        self.apply(instr.INSTR_INIT, [q0]) # Inicializando o qubit
        yield self.run() # Rodando as instruções agendadas acima

<br><br>Como acabamos de fazer para node3a, agora escrevemos um protocolo para node3b. O protocolo irá
- executar o programa (que inicializa o qubit no node3b);
- receber o bit clássico enviado por node3a;
- alterar o estado do qubit.

<span style = "color: orange"> classnetsquid.protocols.nodeprotocols.NodeProtocol(node=None, name=None)</span>

In [6]:
class node3bProtocol(NodeProtocol):
    def run(self):
        port_cin = self.node.ports["cin_3b"]
        node3bprog = node3bProgram()
        
        yield self.node.qmemory.execute_program(node3bprog) # qubit tem que ser inicializado
        
        #Aguardando a mensagem chegar aqui
        expr = yield(self.await_port_input(port_cin))
        message = self.node.ports["cin_3b"].rx_input()
        
        if message.items[0] == 1 :
            self.node.qmemory.execute_instruction(instr.INSTR_H,[0]) # Aplica a porta H
            print("aplicando porta H")
        if message.items[0] == 0 :
            self.node.qmemory.execute_instruction(instr.INSTR_X,[0]) # Aplica a porta X
            print("aplicando porta X")

<br><br> Agora, nós podemos rodar os dois protocolos em nossos dois nós

In [7]:
node3aprot = node3aProtocol(node3a)
node3bprot = node3bProtocol(node3b)

node3aprot.start() # Inicia o protocolo no node3a
node3bprot.start() # Inicia o protocolo no node3b

ns.sim_run() # Inicia a simulação
ns.sim_time

measurement outcome 0
aplicando porta X


<function netsquid.util.simtools.sim_time(magnitude=1)>

## Sugestões de Prática:

Combine o que praticamos nos últimos exemplos:

- Crie um processador quântico com duas posições de memória e atribua-o ao nó A;
- Crie um processador quântico com duas posições de memória e atribua-o ao nó B.

Use programas quânticos e protocolos de nós para

- Inicialize os dois qubits no nó A;
- Aplique a porta H ao segundo qubit;
- Meça o segundo qubit;
- Envie o primeiro qubit para nodeB;
- Envie o resultado da medição para o nó B.
use o resultado da medição para decidir como alterar o estado do qubit no nó B